In [ ]:
!pip install llama-cpp-python
!pip install spacy
!python - m spacy download en_core_web_sm

  Using cached llama_cpp_python-0.1.77.tar.gz (1.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.1-py3-none-any.whl (45 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [114 lines of output]
      
      
      --------------------------------------------------------------------------------
      -- Trying 'Ninja' generator
      --------------------------------
      ---------------------------
      ----------------------
      -----------------
      ------------
      -------
      --
      CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
        Compatibility with CMake < 3.5 will be removed from a future version of
        CMake.
      
        Update the VERSION argument <min> value or use a ...<max> suffix to tell
        CMake that th

In [ ]:
from llama_cpp import Llama
from tqdm import tqdm

import re
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
MODEL_PATH = "../../../llama.cpp/models/vicuna-7b-1.1.ggmlv3.q4_0.bin"
llm = Llama(model_path=MODEL_PATH, verbose=False)

In [ ]:
def style_transfer(style, text, llm, nlp, max_tokens, 
                   temperature = 0.8, top_p=0.95,
                   top_k=40, reset=True):
    doc = nlp(text)
    sents = [s.text for s in doc.sents]
    start = 0
    end = len(sents)
    step = 6
    
    output_list = []
    for i in tqdm(range(start, end, step)):
        prompt_sentences = " ".join(sents[i:i+step]).replace('\n', '')
        prompt = f"""Q: Can you re-write the following in the style of {style}: 
        
        '{prompt_sentences}'

        A:
        """
        output = llm(prompt, echo=False, 
                     max_tokens = max_tokens,
                     temperature = temperature, 
                     top_p=top_p, top_k=top_k)
        output_list.append((prompt, output))
        if reset:
            llm.reset()
    return output_list

In [ ]:
text = """
Different groups of people cycle through the farms in neighborhoods outside downtown, ready to provide food for thousands of people occupying Woodruff Park. 
A warehouse on the west side has trucks and teams to drive to abandoned hotels and industrial waste facilities, gathering “raw” material — metal, lumber, kitchen equipment — that can be used to build brick ovens and fix up the new building. 
A partisan cafe downtown functions as an entry point for visitors and newcomers, as well as a drop-in point for insurgents from around the state, the region, the country, and even the world. The dance club lets people in to blend with the crowd after a rowdy demo while giving them a way to blow off some steam. 
Pirate radio transmitters broadcast from secret locations outside of the city to spread sedition and heresy into the heart of a great metropolis. University copy machines are hacked for free prints for this weekend’s assembly — the print shop is already running overtime. 
A friend walks out of the store with a backpack full of goods and a knowing wink. Doctors and herbalists are at hand, equipped to deal with any injuries that might ensue from tonight’s riot, well trained from treating common ailments and injuries. 
The family lake house is repurposed to sleep a hundred for a summer strategy meeting. Slowly, something is growing.
"""

In [ ]:
outputs = style_transfer("Jean Baudrillard", text, llm, nlp, max_tokens=512)

In [ ]:
def extract_choices(outputs):
    choices = []
    for o in outputs:
        choices.append(o[1]['choices'][0]['text'].replace("\n", ""))
    return " ".join(choices).replace("\n", "")

In [ ]:
extract_choices(outputs)